Importamos los modulos necesarios, pandas para trabajar  y numpy para trabajar nuestros datos, train test para elegir datos de entrenamiento y prueba, logisticRegression para realizar el modelo ya que es binario elejimos este modelo, y las metricas para revisar nuestra prediccion 

In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Cagamos nuestros datos con pandas dataframe

In [350]:
df=pd.read_csv('E-Commerce_train.csv',sep=';')

Obervamos el resltado del Dataframe

In [351]:
df

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8994,8995,D,Ship,3,1,217,3,low,F,1,4177,1
8995,8996,F,Ship,5,3,232,3,medium,M,3,4526,0
8996,8997,A,Ship,4,5,260,3,medium,F,6,4221,0
8997,8998,B,Ship,4,2,184,3,medium,F,5,5931,1


In [352]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='if_binary', sparse = False) # Instanciamos el encoder por binary feature

ohe.fit(df[['Gender']]) # 

df['Gender'] = ohe.transform(df[['Gender']]) # Encodeamos Street

df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,0.0,44,1233,1
1,2,F,Flight,4,5,216,2,low,1.0,59,3088,1
2,3,A,Flight,2,2,183,4,low,1.0,48,3374,1
3,4,B,Flight,3,3,176,4,medium,1.0,10,1177,1
4,5,C,Flight,2,2,184,3,medium,0.0,46,2484,1


In [353]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8999 entries, 0 to 8998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   8999 non-null   int64  
 1   Warehouse_block      8999 non-null   object 
 2   Mode_of_Shipment     8999 non-null   object 
 3   Customer_care_calls  8999 non-null   int64  
 4   Customer_rating      8999 non-null   int64  
 5   Cost_of_the_Product  8999 non-null   int64  
 6   Prior_purchases      8999 non-null   int64  
 7   Product_importance   8999 non-null   object 
 8   Gender               8999 non-null   float64
 9   Discount_offered     8999 non-null   int64  
 10  Weight_in_gms        8999 non-null   int64  
 11  Reached.on.Time_Y.N  8999 non-null   int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 843.8+ KB


In [257]:
# Using make_column_transformer to One-Hot Encode
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from seaborn import load_dataset
import pandas as pd


transformer = make_column_transformer(
    (OneHotEncoder(), ['Warehouse_block']),
    remainder='passthrough')

transformed = transformer.fit_transform(df)
t_df = pd.DataFrame(
    transformed, 
    columns=transformer.get_feature_names_out()
)

In [263]:
# One-hot encoding a single column
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset

ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[['Product_importance']])
print(transformed.toarray())
df[ohe.categories_[0]] = transformed.toarray()
print(df.head())

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]
   ID Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0   1               D           Flight                    4                2   
1   2               F           Flight                    4                5   
2   3               A           Flight                    2                2   
3   4               B           Flight                    3                3   
4   5               C           Flight                    2                2   

   Cost_of_the_Product  Prior_purchases Product_importance  Gender  \
0                  177                3                low     0.0   
1                  216                2                low     1.0   
2                  183                4                low     1.0   
3                  176                4             medium     1.0   
4                  184                3             medium     0.0   

   Discount_offered 

Revisamos valores flatantes del dataframe

In [354]:
df

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,0.0,44,1233,1
1,2,F,Flight,4,5,216,2,low,1.0,59,3088,1
2,3,A,Flight,2,2,183,4,low,1.0,48,3374,1
3,4,B,Flight,3,3,176,4,medium,1.0,10,1177,1
4,5,C,Flight,2,2,184,3,medium,0.0,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8994,8995,D,Ship,3,1,217,3,low,0.0,1,4177,1
8995,8996,F,Ship,5,3,232,3,medium,1.0,3,4526,0
8996,8997,A,Ship,4,5,260,3,medium,0.0,6,4221,0
8997,8998,B,Ship,4,2,184,3,medium,0.0,5,5931,1


In [355]:
df.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64

In [356]:
df.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,8999.00000,8999.000000,8999.000000,8999.000000,8999.000000,8999.000000,8999.000000,8999.000000,8999.000000
mean,4500.00000,4.015113,2.994999,208.561507,3.511501,0.495499,15.097678,3584.364596,0.635515
std,2597.93187,1.138934,1.413143,48.289172,1.532074,0.500008,17.400040,1614.750194,0.481312
min,1.00000,2.000000,1.000000,96.000000,2.000000,0.000000,1.000000,1001.000000,0.000000
25%,2250.50000,3.000000,2.000000,168.000000,3.000000,0.000000,4.000000,1857.000000,0.000000
50%,4500.00000,4.000000,3.000000,211.000000,3.000000,0.000000,8.000000,4046.000000,1.000000
75%,6749.50000,5.000000,4.000000,249.500000,4.000000,1.000000,19.000000,4999.500000,1.000000
max,8999.00000,7.000000,5.000000,310.000000,10.000000,1.000000,65.000000,7846.000000,1.000000


Revisamos la distribucion de la variable ojetivo

In [379]:
df['Reached.on.Time_Y.N'].value_counts()

1    5719
0    3280
Name: Reached.on.Time_Y.N, dtype: int64

In [358]:
df.tail()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
8994,8995,D,Ship,3,1,217,3,low,0.0,1,4177,1
8995,8996,F,Ship,5,3,232,3,medium,1.0,3,4526,0
8996,8997,A,Ship,4,5,260,3,medium,0.0,6,4221,0
8997,8998,B,Ship,4,2,184,3,medium,0.0,5,5931,1
8998,8999,C,Ship,5,4,145,4,low,1.0,4,4790,0


In [359]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8999 entries, 0 to 8998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   8999 non-null   int64  
 1   Warehouse_block      8999 non-null   object 
 2   Mode_of_Shipment     8999 non-null   object 
 3   Customer_care_calls  8999 non-null   int64  
 4   Customer_rating      8999 non-null   int64  
 5   Cost_of_the_Product  8999 non-null   int64  
 6   Prior_purchases      8999 non-null   int64  
 7   Product_importance   8999 non-null   object 
 8   Gender               8999 non-null   float64
 9   Discount_offered     8999 non-null   int64  
 10  Weight_in_gms        8999 non-null   int64  
 11  Reached.on.Time_Y.N  8999 non-null   int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 843.8+ KB


In [360]:

X=df.drop(columns=['Reached.on.Time_Y.N','Mode_of_Shipment','Product_importance','Warehouse_block'], axis=1)
Y=df['Reached.on.Time_Y.N']

In [361]:
X.shape

(8999, 8)

Seleccionamos nuestros datos de entrenamiento y de test y los dividimos de manera aeatoria

In [362]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y,random_state=2)

In [363]:
X.shape, X_train.shape, X_test.shape, 

((8999, 8), (7199, 8), (1800, 8))

In [364]:
modelo1=LogisticRegression()

In [392]:
modelo1.fit(X_train,Y_train)

LogisticRegression()

In [393]:
X_train_prediction=modelo1.predict(X_train)
Precision_Datos_entrenamiento=accuracy_score(X_train_prediction,Y_train)
from sklearn.metrics import recall_score
Calculo_Recallo=recall_score(X_train_prediction,Y_train)

In [400]:
X_train

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Gender,Discount_offered,Weight_in_gms
2983,2984,4,3,222,4,1.0,49,3697
1834,1835,5,5,234,2,1.0,44,2548
4811,4812,6,5,245,4,0.0,8,1403
1456,1457,3,5,156,3,1.0,44,1424
5414,5415,5,2,247,3,0.0,3,4698
...,...,...,...,...,...,...,...,...
4832,4833,4,5,140,3,0.0,8,4956
2741,2742,4,2,171,3,0.0,40,1021
8748,8749,4,5,258,3,1.0,8,4046
3957,3958,3,3,250,3,0.0,5,4625


In [395]:
Precision_Datos_entrenamiento,Calculo_Recallo

(0.6855118766495346, 0.7613662067405564)

In [396]:
X_test_prediction=modelo1.predict(X_test)
Precision_Datos_test=accuracy_score(X_test_prediction,Y_test)
Calculo_Recallo=recall_score(X_test_prediction,Y_test)

In [397]:
Precision_Datos_entrenamiento,Calculo_Recallo

(0.6855118766495346, 0.7568075117370892)

In [398]:
dftest=pd.read_csv('E-Commerce_test.csv',sep=';')

In [399]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='if_binary', sparse = False) # Instanciamos el encoder por binary feature

ohe.fit(dftest[['Gender']]) # 

dftest['Gender'] = ohe.transform(dftest[['Gender']]) # Encodeamos Street

dftest.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9000,F,Ship,5,1,189,2,medium,1.0,10,5248
1,9001,D,Ship,5,3,150,2,high,0.0,9,4446
2,9002,F,Ship,3,3,200,3,low,1.0,3,5739
3,9003,A,Ship,5,1,133,2,medium,1.0,1,4643
4,9004,B,Ship,4,2,260,3,low,1.0,4,5504


In [404]:
dftest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2000 non-null   int64  
 1   Warehouse_block      2000 non-null   object 
 2   Mode_of_Shipment     2000 non-null   object 
 3   Customer_care_calls  2000 non-null   int64  
 4   Customer_rating      2000 non-null   int64  
 5   Cost_of_the_Product  2000 non-null   int64  
 6   Prior_purchases      2000 non-null   int64  
 7   Product_importance   2000 non-null   object 
 8   Gender               2000 non-null   float64
 9   Discount_offered     2000 non-null   int64  
 10  Weight_in_gms        2000 non-null   int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 172.0+ KB


In [334]:
# One-hot encoding a single column
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset

ohe = OneHotEncoder()
transformed = ohe.fit_transform(dftest[['Product_importance']])
print(transformed.toarray())
dftest[ohe.categories_[0]] = transformed.toarray()
print(dftest.head())

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
     ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0  9000               F             Ship                    5   
1  9001               D             Ship                    5   
2  9002               F             Ship                    3   
3  9003               A             Ship                    5   
4  9004               B             Ship                    4   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                1                  189                2             medium   
1                3                  150                2               high   
2                3                  200                3                low   
3                1                  133                2             medium   
4                2                  260                3                low   

   Gender  Discount_offered  ...  Road  Ship    A    B  

In [405]:
X1=dftest.drop(columns=['Mode_of_Shipment','Product_importance','Warehouse_block'], axis=1)


In [391]:
X1.shape

(2000, 8)

In [407]:
X1prediction=modelo1.predict(X1)

In [408]:
X1prediction

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [376]:
type(X1prediction)

numpy.ndarray

In [377]:
datasalida = pd.DataFrame(X1prediction)

In [410]:
datasalida.to_csv('salida.csv', index=False)

In [409]:
datasalida[0].value_counts()

0    1956
1      44
Name: 0, dtype: int64

In [388]:

sss = pd.DataFrame(X_train_prediction)

In [389]:
sss[0].value_counts()

1    4421
0    2778
Name: 0, dtype: int64